In [101]:
import numpy as np

def bisection_method(f, a, b, tol):
    if f(a)*f(b) > 0:
        #end function, no root.
        print("No root found.")
    else:
        while (b - a)/2.0 > tol:
            midpoint = (a + b)/2.0
            if f(midpoint) == 0:
                return(midpoint) #The midpoint is the x-intercept/root.
            elif f(a)*f(midpoint) < 0: # Increasing but below 0 case
                b = midpoint
            else:
                a = midpoint
        return(midpoint)

* 1 MWt
* Approx size 45.72 cm by 60.96 cm.
* 570C: liquid metal coolat temperatures.

* 600 KWt
* 211 1.4224cm diameter ZrH uranium fuel rods: Uranium Zr allow, hydrided.
* Be as reflector.
* 22.86 cm diameter by 50.8 cm high Stainless Steel vessel

* NaK as coolant
* 650C operational temp
* 20.3 cm (8in) diameter by 30.5 cm (12in)

SNAP 2DR:
* 50 kWt
* 650 C

SNAP 10A:
* 567C
* 35 kWt/960 lb

S8DR reactor:
* 600 kWt
* 706C
* 211 cylindrical rods of uranium Zr alloy, hybrided: 
    - 6.1e22 at/cc of Hydrogen
    - 10.5 wt% U235
    - 0.53in (1.35 cm) diam, 16.8in (42.8 cm) length
    - Clad: Hastelloy N
* Be reflector: 5in (12.7 cm)
* overall size 20in (50.8 cm) in diameter and 36.5in (92.7cm) in length
* weight: 950 lb

[1] Partel G.A., Space Engineering, 1970.

$\gamma$-ZrH:
$\rho$ = 5.806 g/cc

$\delta$-ZrH$_{1.5}$:
$\rho$ = 5.624 g/cc

[2] Weck et al. Mechanical properties of zirconium alloys and zirconium hydrides
predicted from density functional perturbation theory. 2015.

# Materials definition: atomic fractions

In [134]:
#U2CO3
NU = 2/6
NC = 1/6
NO = 3/6

e = 0.2
A = e/(1-e)*238/235
NU235 = A*NU/(1+A)
NU238 = NU - NU235
print('UCO:')
print('NU235: ', NU235)
print('NU238: ', NU238)
print('NO: ', NO)
print('NC: ', NC)
# print(NU235+NU238+NC+NO)

NZr = 1
NH = 1.5
Nt = NZr + NH
print('\nZrH:')
print('NH: ', NH/Nt)

Nzrtot = 51.45+11.22+17.15+17.38
NZr90 = NZr/Nt*51.45/Nzrtot
NZr91 = NZr/Nt*11.22/Nzrtot
NZr92 = NZr/Nt*17.15/Nzrtot
NZr94 = NZr/Nt*17.38/Nzrtot
print('NZr90: ', NZr90)
print('NZr91: ', NZr91)
print('NZr92: ', NZr92)
print('NZr94: ', NZr94)

T = 600  # C
T = 600+273  #K
print('\nTemp: ', T)

UCO:
NU235:  0.06734578381437464
NU238:  0.26598754951895864
NO:  0.5
NC:  0.16666666666666666

ZrH:
NH:  0.6
NZr90:  0.21172839506172844
NZr91:  0.04617283950617285
NZr92:  0.0705761316872428
NZr94:  0.07152263374485597

Temp:  873


# Different Geometries

### Bare reactor

In [146]:
def calculateM(R):
    '''
    Calculates reactor mass (M)for a given radius(R).
    The reactor does not have a reflector.

    Parameters:
    -----------
    R: [float]
        Radius of the active core [cm]
    Returns:
    --------
    M: [float]
        Total mass [g]
    M_zrh: [float]
        ZrH total mass [g]
    M_uco: [float]
        UCO total mass [g]
    '''
    r_uco = 0.02125  # [cm]
    r_buf = 0.03125  # [cm]
    r_ipyc = 0.03475  # [cm]
    r_sic = 0.03825  # [cm]
    r_opyc = 0.04225  # [cm]

    rho_zrh = 5.624  # [g/cm3]
    rho_uco = 10.5  # [g/cm3]
    rho_buf = 1.0  # [g/cm3]
    rho_ipyc = 1.9  # [g/cm3]
    rho_sic = 3.2  # [g/cm3]
    rho_opyc = 1.9  # [g/cm3]

    H = 2*R
    V_tot = np.pi*R**2*H
    V_triso = 0.42*V_tot
    V_zrh = V_tot - V_triso

    V_uco = V_triso * (r_uco/r_opyc)**3
    V_buf = V_triso * ((r_buf/r_opyc)**3 - (r_uco/r_opyc)**3)
    V_ipyc = V_triso * ((r_ipyc/r_opyc)**3 - (r_buf/r_opyc)**3)
    V_sic = V_triso * ((r_sic/r_opyc)**3 - (r_ipyc/r_opyc)**3)
    V_opyc = V_triso * (1 - (r_sic/r_opyc)**3)

    v_triso = 4/3*np.pi*r_opyc**3
    nparticles = V_triso/v_triso
    # print(nparticles)

    M_zrh = rho_zrh * V_zrh
    M_uco = rho_uco * V_uco
    M_buf = rho_buf * V_buf
    M_ipyc = rho_ipyc * V_ipyc
    M_sic = rho_sic * V_sic
    M_opyc = rho_opyc * V_opyc

    M = M_zrh + M_uco + M_buf + M_ipyc + M_sic + M_opyc
    return M, M_zrh, M_uco


def findR(R):
    return calculateM(R)[0] - 3000e3

R = bisection_method(findR, 30, 60, 0.001)
print('R: {0} cm'.format(R))
print('H: {0} cm'.format(2*R))

R: 47.2869873046875 cm
H: 94.573974609375 cm


In [149]:
R = 47.2

M, M_zrh, M_uco = calculateM(R)
print('M: ', M/1e3)
print('M_zrh: ', M_zrh/1e3)
print('M_uco: ', M_uco/1e3)

M:  2983.712670882685
M_zrh:  2155.1582727412433
M_uco:  370.7158178119561


### Only radial reflector

Be Metal:
density (600C) 1.8 g/cm3
Try thickness of 13 cm

[3] Beestom, BERYLLIUM METAL AS A NEUTRON MODERATOR AND REFLECTOR MATERIAL. 1970.

In [150]:
def calculateM2(R):
    '''
    Calculates reactor mass (M) for a given radius(R).
    The reactor has a radial reflector.

    Parameters:
    -----------
    R: [float]
        Radius of the active core [cm]
    Returns:
    --------
    M: [float]
        Total mass [g]
    M_zrh: [float]
        ZrH total mass [g]
    M_uco: [float]
        UCO total mass [g]
    M_be: [float]
        Be total mass [g]
    '''
    r_uco = 0.02125  # [cm]
    r_buf = 0.03125  # [cm]
    r_ipyc = 0.03475  # [cm]
    r_sic = 0.03825  # [cm]
    r_opyc = 0.04225  # [cm]

    rho_zrh = 5.624  # [g/cm3]
    rho_uco = 10.5  # [g/cm3]
    rho_buf = 1.0  # [g/cm3]
    rho_ipyc = 1.9  # [g/cm3]
    rho_sic = 3.2  # [g/cm3]
    rho_opyc = 1.9  # [g/cm3]

    H = 2*R
    V_tot = np.pi*R**2*H
    V_triso = 0.42*V_tot
    V_zrh = V_tot - V_triso

    V_uco = V_triso * (r_uco/r_opyc)**3
    V_buf = V_triso * ((r_buf/r_opyc)**3 - (r_uco/r_opyc)**3)
    V_ipyc = V_triso * ((r_ipyc/r_opyc)**3 - (r_buf/r_opyc)**3)
    V_sic = V_triso * ((r_sic/r_opyc)**3 - (r_ipyc/r_opyc)**3)
    V_opyc = V_triso * (1 - (r_sic/r_opyc)**3)

    v_triso = 4/3*np.pi*r_opyc**3
    nparticles = V_triso/v_triso
    # print(nparticles)
    
    M_zrh = rho_zrh * V_zrh
    M_uco = rho_uco * V_uco
    M_buf = rho_buf * V_buf
    M_ipyc = rho_ipyc * V_ipyc
    M_sic = rho_sic * V_sic
    M_opyc = rho_opyc * V_opyc
    
    rho_be = 1.8
    r_be = 13
    V_be = np.pi*((r_be+R)**2-R**2)*H
    M_be = rho_be * V_be

    M = M_zrh + M_uco + M_buf + M_ipyc + M_sic + M_opyc + M_be
    return M, M_zrh, M_uco, M_be


def findR2(R):
    return calculateM2(R)[0] - 3000e3


R = bisection_method(findR2, 30, 60, 0.001)
print('R: {0} cm'.format(R))
print('H: {0} cm'.format(2*R))

R: 43.6322021484375 cm
H: 87.264404296875 cm


### Radial + Top + Bottom reflector

In [154]:
def calculateM3(R):
    '''
    Calculates reactor mass (M) for a given radius(R).
    The reactor has a radial, bottom, and top reflector.
    Radial reflector thickness: 13 cm.
    Bottom and top reflector thickness: 10 cm.

    Parameters:
    -----------
    R: [float]
        Radius of the active core [cm]
    Returns:
    --------
    M: [float]
        Total mass [g]
    M_zrh: [float]
        ZrH total mass [g]
    M_uco: [float]
        UCO total mass [g]
    M_be: [float]
        Be total mass [g]
    '''
    r_uco = 0.02125  # [cm]
    r_buf = 0.03125  # [cm]
    r_ipyc = 0.03475  # [cm]
    r_sic = 0.03825  # [cm]
    r_opyc = 0.04225  # [cm]

    rho_zrh = 5.624  # [g/cm3]
    rho_uco = 10.5  # [g/cm3]
    rho_buf = 1.0  # [g/cm3]
    rho_ipyc = 1.0  # [g/cm3]
    rho_sic = 3.2  # [g/cm3]
    rho_opyc = 1.0  # [g/cm3]

    V_tot = 2*np.pi*R**3
    V_triso = 0.42*V_tot
    V_zrh = V_tot - V_triso

    V_uco = V_triso * (r_uco/r_opyc)**3
    V_buf = V_triso * ((r_buf/r_opyc)**3 - (r_uco/r_opyc)**3)
    V_ipyc = V_triso * ((r_ipyc/r_opyc)**3 - (r_buf/r_opyc)**3)
    V_sic = V_triso * ((r_sic/r_opyc)**3 - (r_ipyc/r_opyc)**3)
    V_opyc = V_triso * (1 - (r_sic/r_opyc)**3)

    v_triso = 4/3*np.pi*r_opyc**3
    nparticles = V_triso/v_triso
    # print(nparticles)

    M_zrh = rho_zrh * V_zrh
    M_uco = rho_uco * V_uco
    M_buf = rho_buf * V_buf
    M_ipyc = rho_ipyc * V_ipyc
    M_sic = rho_sic * V_sic
    M_opyc = rho_opyc * V_opyc
    
    rho_be = 1.8
    r_be = 13  # cm, radial reflector tickness
    V_be = np.pi*((r_be+R)**2-R**2)*2*R
    t_be = 10  # cm, top and bottom reflector tickness
    V_be += np.pi*((r_be+R)**2)*(2*t_be)
    M_be = rho_be * V_be

    M = M_zrh + M_uco + M_buf + M_ipyc + M_sic + M_opyc + M_be
    return M, M_zrh, M_uco, M_be


def findR3(R):
    return calculateM3(R)[0] - 3000e3


R = bisection_method(findR3, 30, 60, 0.001)
print('R: {0} cm'.format(R))

R: 42.1563720703125 cm


In [155]:
R = 42

M, M_zrh, M_uco, M_be = calculateM3(R)
print('M: ', M/1e3)
print('M_zrh: ', M_zrh/1e3)
print('M_uco: ', M_uco/1e3)
print('M_be: ', M_be/1e3)

M:  2971.2305999485584
M_zrh:  1518.4519202803604
M_uco:  261.19387729183944
M_be:  941.10554840585
